<a href="https://colab.research.google.com/github/aayushmanace/Computer-Vision/blob/main/20004_Aayushman_Assignment1_DrVaibhav.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import glob
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from torch.utils.data import DataLoader, Dataset
import torch

In [ ]:
#!pip install pygad

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/Dataset"

In [ ]:
r = os.listdir(path)
r[0]

'Main_Dataset'

In [ ]:
csv_files = glob.glob(os.path.join(path, "Main_dataset")+"/*.csv")

for file in csv_files:
    # Get the filename without the path or extension
    filename = os.path.splitext(os.path.basename(file))[0]

    # Read the CSV file into a pandas DataFrame with the same name as the file
    globals()[filename] = pd.read_csv(file)

In [ ]:
path2 = "/content/drive/MyDrive/Dataset/Prepared_Dataset"

In [ ]:
data_dir = path2
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
val_dir = os.path.join(data_dir, 'val')

In [ ]:
# Define the image and mask directories
image_dir = 'images'
mask_dir = 'masks'

# Define a function to get the file paths for all the images and masks in a directory
def get_image_mask_paths(dir_path):
    # Get the paths for all the image files
    image_paths = [os.path.join(dir_path, image_dir, f) for f in os.listdir(os.path.join(dir_path, image_dir))]
    
    # Get the corresponding paths for the mask files
    mask_paths = [os.path.join(dir_path, mask_dir, os.path.splitext(f)[0] + '.png') for f in os.listdir(os.path.join(dir_path, image_dir)) ]


    return image_paths, mask_paths

# Get the file paths for all the images and masks in the train, test, and val directories
train_image_paths, train_mask_paths = get_image_mask_paths(train_dir)
test_image_paths, test_mask_paths  = get_image_mask_paths(test_dir)
val_image_paths, val_mask_paths = get_image_mask_paths(val_dir)


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, image_paths, mask_paths,transform = None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Load image and mask
        image = Image.open(self.image_paths[idx]).convert("RGB")
        mask = Image.open(self.mask_paths[idx]).convert("RGB")

         # Convert PIL images to PyTorch tensors
        x = torch.Tensor(np.array(image))
        image = (x - x.min()) / (x.max() - x.min())
        y = torch.Tensor(np.array(mask))
        mask = (y - y.min())/(y.max() - y.min())

        return image, mask

In [ ]:
train_dataset = CustomDataset(train_image_paths, train_mask_paths)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)

val_dataset = CustomDataset(val_image_paths[:24], val_mask_paths[:24])
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=8, shuffle=True)


In [ ]:
import torch 
import torch.nn as nn
import torchvision.transforms.functional as TF
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp


##Define the neural network 
class DoubleConv(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(DoubleConv,self).__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
    )

  def forward(self, x):
    return self.conv(x)


class UNet(nn.Module):
  def __init__(self, in_channels=3, out_channels=3, features =[64, 128, 256, 512]):
    super(UNet, self).__init__()
    self.ups = nn.ModuleList()
    self.downs = nn.ModuleList()
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    #down part of UNet
    for feature in features:
      self.downs.append(DoubleConv(in_channels, feature))
      in_channels = feature

    #upsample part of UNet
    for feature in reversed(features):
      self.ups.append(
          nn.ConvTranspose2d(
              feature*2, feature, kernel_size=2, stride = 2
          )
      )
      self.ups.append(DoubleConv(feature*2, feature))

    self.bottleneck = DoubleConv(features[-1], features[-1]*2)
    self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size = 1)

  def forward(self, x):
    skip_connections = []

    for down in self.downs:
      x = down(x)
      skip_connections.append(x)
      x = self.pool(x)

    x = self.bottleneck(x)
    skip_connections = skip_connections[::-1]

    for idx in range(0, len(self.ups), 2):
      x = self.ups[idx](x)
      skip_connection = skip_connections[idx//2]

      if x.shape != skip_connection.shape:
        x = TF.resize(x, size = skip_connection.shape[2:])
        
      concat_skip = torch.cat((skip_connection, x), dim=1)
      x = self.ups[idx+1](concat_skip)

    return self.final_conv(x)


In [ ]:
train_on_gpu = torch.cuda.is_available()
model =  UNet()
if train_on_gpu:
    model.cuda()


In [ ]:
train_on_gpu

True

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

num_epoch = 100

print("Training Begins")
for epoch in range(num_epoch):
  running_loss = 0.0
  for batch_idx,(data, target) in enumerate(train_loader):
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    optimizer.zero_grad()

    output = model.forward(data.permute(0,3,1,2))

    loss = criterion(output, target.permute(0,3,1,2))
    
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
  if epoch%10==0 or epoch==num_epoch-1:
    print('Epoch [%d/%d], Loss: %.4f' % (epoch+1, num_epoch, running_loss))

def validate(model, val_loader, criterion):
    model.eval()
    val_loss = 0
    val_acc = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            if train_on_gpu:
                inputs, labels = inputs.cuda(), labels.cuda()
            outputs = model(inputs.permute(0,3,1,2))
            loss = criterion(outputs, labels.permute(0,3,1,2))
            val_loss += loss.item() * inputs.size(0)
            #_, preds = torch.max(outputs, 1)
            #val_acc += torch.sum(preds == labels.data)
    val_loss = val_loss / len(val_loader.dataset)
    #val_acc = val_acc / len(val_loader.dataset)
    print('Validation Loss: {:.4f}'.format(val_loss))

validate(model, val_loader, criterion )

Training Begins
Epoch [1/100], Loss: 184.7562
Epoch [11/100], Loss: 175.6416
Epoch [21/100], Loss: 171.4307
Epoch [31/100], Loss: 170.5258
Epoch [41/100], Loss: 169.9754
Epoch [51/100], Loss: 167.3105
Epoch [61/100], Loss: 166.2158
Epoch [71/100], Loss: 163.6041
Epoch [81/100], Loss: 161.7247
Epoch [91/100], Loss: 163.1822
Epoch [100/100], Loss: 158.1766
Validation Loss: 1.4316


## Testing the Model Images

In [ ]:
iterator = iter(train_loader)
sample = next(iterator)
a,b = sample

In [ ]:
a = a.cuda()
c = model(a.permute(0,3,1,2))
t = c.permute(0,2,3,1)
t  = t.cpu()
t = t.detach().numpy()
a = a.cpu()
a = a.detach().numpy()

In [ ]:
t.shape[0]

8

In [ ]:
fig, ax = plt.subplots(nrows=8, ncols=3,figsize=(10,20))
for i in range(4):
  ax[i, 0].imshow(np.clip(a[i], 0, 1))
  ax[i, 0].axis('off')
  ax[i, 1].imshow(np.clip(t[i], 0, 1))
  ax[i, 1].axis('off')
  ax[i, 2].imshow(np.clip(b[i], 0, 1))
  ax[i, 2].axis('off')
plt.show()

NameError: ignored

In [ ]:
for data, target in val_loader:
  print(data.shape)

torch.Size([8, 120, 120, 3])
torch.Size([8, 120, 120, 3])
torch.Size([8, 120, 120, 3])


In [ ]:
for data, target in val_loader:
  print(data.permute(0,3,1,2).shape)
  if train_on_gpu:
    data, target = data.cuda(), target.cuda()
  model.forward(data.permute(0,3,1,2)).shape

torch.Size([8, 3, 120, 120])
torch.Size([8, 3, 120, 120])
torch.Size([8, 3, 120, 120])


In [ ]:
a.shape, b.shape

((8, 120, 120, 3), torch.Size([8, 120, 120, 3]))

In [ ]:
model.forward(a.permute(0,3,1,2))

AttributeError: ignored

In [ ]:
a = torch.randn(8,3,120,120)

In [ ]:
a.permute(0,3,2,1).permute(0,3,1,2).shape